# Required Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Load image

**Get respective folder_path and filename for saving processed images**

In [ ]:
def load_image(path):                                                      
    image = cv2.imread(path)                                # Load the eye image and convert to grayscale
    image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)      # Convert to grayscale

    folder_path, filename = os.path.split(path)             # Get the image path

    # Display the grayscale image using Matplotlib
    # plt.imshow(image, cmap='gray')
    # plt.show()

    return image_gs, folder_path, filename

# Pupil Segementation

In [ ]:
def pupil_segmentation(image, threshold_value):
    global eye_folder_path, eye_filename
    
    # Step 1: Inverse the image
    inv_eye_image = cv2.bitwise_not(image)
    
    # Step 2: Apply threshold to the inverse image
    ret, thresh = cv2.threshold(inv_eye_image, threshold_value*255, 255, cv2.THRESH_BINARY)
    
    # Step 3: Erosion with a circular structural element
    kernel_size = (1, 1)  # Adjust the kernel size as needed
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    eroded_img = cv2.erode(thresh, kernel, iterations=50)
    
    # Step 4: Finding the largest connected component
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(eroded_img, connectivity=8)
    
    # Get the index of the component with the largest area
    max_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1
    max_area = stats[max_label, cv2.CC_STAT_AREA]

    # Get the x, y coordinates of the centroid of the largest component
    x, y = map(int, centroids[max_label])

    # Calculate the radius
    radius = int(np.sqrt(max_area / np.pi)) + 2
   
    # Draw a thick red circle around the largest component
    img_circle = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    cv2.circle(img_circle, (x, y), radius, (255, 0, 0), 2)
    
    # Save the Image
    new_filename = eye_filename.replace('.bmp', '_1_Pupil_Segmented.bmp')
    new_image_path = os.path.join(eye_folder_path, new_filename)
    cv2.imwrite(new_image_path, cv2.cvtColor(img_circle, cv2.COLOR_RGB2BGR))
    
    # Display the final image with the red circle
    #plt.imshow(img_circle)
    #plt.show()

    return x, y, radius

# Iris Segmentation

**Gamma Correction**

In [ ]:
def gamma_correction(gamma, image, radius):
    
    img_gamma = np.power(image / 255, gamma) * 255
    img_gamma = np.array(img_gamma, dtype=np.uint8)
    
    # Step 2: Process with a median filter
    img_median = cv2.medianBlur(img_gamma, 5)
    
    # Step 3: Circular Hough Transform
    circles = cv2.HoughCircles(
        img_median, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
        param1=50, param2=30, minRadius=radius, maxRadius=3 * radius)

    return circles

**Iris Segmentation**

In [ ]:
def iris_seg(g, image, x, y, radius, t, circles):     # t = threshold, g = gamma
    
    global eye_folder_path, eye_filename

    if circles is not None:
        circles = np.uint16(np.around(circles))
        img_circle = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        for i in circles[0, :]:
            cv2.circle(img_circle, (i[0], i[1]), i[2], (255, 0, 0), 2)   # draw the outer circle
            cv2.circle(img_circle, (i[0], i[1]), 2, (255, 0, 0), 3)      # draw the center of the circle
        #plt.imshow(img_circle)
        
        # Step 4: Filter circles with offset from iris < 20
        iris_circles = [i for i in circles[0, :] if np.sqrt((i[0] - x) ** 2 + (i[1] - y) ** 2) < 20]
        iris_circles = np.array(iris_circles)
        img_circle = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        for i in iris_circles:
            cv2.circle(img_circle, (i[0], i[1]), i[2], (255, 0, 0), 2)    # draw the outer circle
            cv2.circle(img_circle, (i[0], i[1]), 2, (255, 0, 0), 3)       # draw the center of the circle
        #plt.imshow(img_circle)
        
        # Step 5: Choose the circle with the largest radius
        if(len(iris_circles) == 0):
            iris_circle = [x, y, int(2.2*radius)]
        else:
            iris_circle = iris_circles[np.argmax(iris_circles[:, 2])]

    else:
        iris_circle = [x, y, int(2.2*radius)]
        
    img_circle = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)    # draw the outer circle
    cv2.circle(img_circle, (iris_circle[0], iris_circle[1]), iris_circle[2], (255, 0, 0), 2)
    
    # Draw pupil circle also
    cv2.circle(img_circle, (x, y), radius, (255, 0, 0), 2)
    
    # Save the Image
    new_filename = eye_filename.replace('.bmp', '_2_Iris_Segmented.bmp')
    new_image_path = os.path.join(eye_folder_path, new_filename)
    cv2.imwrite(new_image_path, cv2.cvtColor(img_circle, cv2.COLOR_RGB2BGR))
    #plt.imshow(img_circle)
    
    return iris_circle

# Iris Normalization

In [ ]:
def iris_normalization(image, iris_circle, radius, h, w):    # h = height, w = width
    #print("Image shape:", image.shape)
    global eye_folder_path, eye_filename
    
    # Radius of outer circle : Iris (approx 63)
    r_out = iris_circle[2]
    # Radius of inner circle : Pupil (approx 23)
    r_in = radius

    #print(r_out)
    #print(r_in)
    
    # Step 1: Rubber sheet model
    img_rubber = np.zeros((h, w, 3), np.uint8)
    thetas = np.linspace(0, 2 * np.pi, w)
    
    for i in range(h):
        for j in range(w):
            r = r_in + (r_out - r_in) * i / h
            x = int(r * np.cos(thetas[j]) + iris_circle[0])
            y = int(r * np.sin(thetas[j]) + iris_circle[1])
            #print(f'{x} {y}')
            img_rubber[i, j] = image[y, x]
    
    # Save the Image
    new_filename = eye_filename.replace('.bmp', '_3_Rubber.bmp')
    new_image_path = os.path.join(eye_folder_path, new_filename)
    cv2.imwrite(new_image_path, img_rubber)
    
    #plt.imshow(img_rubber, cmap='gray')

    return img_rubber

# Eyelash Detection

In [ ]:
def eyelash_detection(img_rubber, t):
    global eye_folder_path, eye_filename
    
    # Step 1: Inverse the image and apply threshold
    img_rubber_gs = cv2.cvtColor(img_rubber, cv2.COLOR_BGR2GRAY)
    inv_eye_image = cv2.bitwise_not(img_rubber_gs)
    ret, thresh2 = cv2.threshold(inv_eye_image, t*255, 255, cv2.THRESH_BINARY)
    
    # Step 2: Erosion with a circular structural element
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
    eroded_img2 = cv2.erode(thresh2, kernel, iterations=1)
    
    # Step 3: Remove connected components with the number of pixels less than 10
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(eroded_img2, connectivity=8)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    min_size = 10
    img_cleaned = np.zeros(output.shape, dtype=np.uint8)
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            img_cleaned[output == i + 1] = 255
    #plt.imshow(img_cleaned, cmap='gray')
    
    # Step 4: Dilate the image with a 4x4 square structural element
    kernel = np.ones((4, 4), np.uint8)
    img_dilated = cv2.dilate(img_cleaned, kernel, iterations=1)
    
    # Save the Image
    new_filename = eye_filename.replace('.bmp', '_4_Dilated.bmp')
    new_image_path = os.path.join(eye_folder_path, new_filename)
    cv2.imwrite(new_image_path, img_dilated)
    
    #plt.imshow(img_dilated, cmap='gray')

    return img_dilated

# Parabolic Projection

In [ ]:
def parabolic_projection(img_dilated, img_rubber):
    global eye_folder_path, eye_filename

    # Step 1: Divide the image into left and right halves
    img_left = img_dilated[:, :180].astype(np.uint8)
    img_right = img_dilated[:, 180:].astype(np.uint8)
    
    # Step 2: For the left side, estimate coefficients a, b, c for the parabolic curve
    edges_left = cv2.Canny(img_left, 30, 70, apertureSize=3)
    indices_left = np.where(edges_left != [0])
    x_left = indices_left[1]
    y_left = indices_left[0]
    A_left = np.array([x_left**2, x_left, np.ones(len(x_left))]).T
    B_left = y_left
    a_left, b_left, c_left = np.linalg.lstsq(A_left, B_left, rcond=None)[0]
    
    # Step 3: For the right side, estimate coefficients a, b, c for the parabolic curve
    edges_right = cv2.Canny(img_right, 30, 70, apertureSize=3)
    indices_right = np.where(edges_right != [0])
    x_right = indices_right[1]
    y_right = indices_right[0]
    A_right = np.array([x_right**2, x_right, np.ones(len(x_right))]).T
    B_right = y_right
    a_right, b_right, c_right = np.linalg.lstsq(A_right, B_right, rcond=None)[0]
    
    # Step 4: Change pixel values to the mean value of the image_rubber if they lie under the parabola
    img_rubber_2 = img_rubber.copy()
    mean_val = np.mean(img_rubber_2)
    
    for i in range(0, img_rubber_2.shape[1] // 2):
        y_left = int(a_left * i**2 + b_left * i + c_left)
        y_right = int(a_right * i**2 + b_right * i + c_right)
        
        if y_left < img_rubber_2.shape[0]:
            img_rubber_2[:y_left, i] = mean_val
        
        if y_right < img_rubber_2.shape[0]:
            img_rubber_2[y_right:, i + img_rubber_2.shape[1] // 2] = mean_val
    
    # Save the Image
    new_filename = eye_filename.replace('.bmp', '_5_Parabolic.bmp')
    new_image_path = os.path.join(eye_folder_path, new_filename)
    cv2.imwrite(new_image_path, img_rubber_2)

    #plt.imshow(img_rubber_2, cmap='gray')
    return img_rubber_2

# Histogram Equalization

In [ ]:
def histogram_equalization(img):
    global eye_folder_path, eye_filename
    # Ensure the image is in grayscale
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Perform histogram equalization
    equalized_img = cv2.equalizeHist(img)

    # Save the Image
    new_filename = eye_filename.replace('.bmp', '_6_HistogramEq.bmp')
    new_image_path = os.path.join(eye_folder_path, new_filename)
    cv2.imwrite(new_image_path, equalized_img)

    return equalized_img

# Main Function

In [ ]:
folder_path = '../DIP-Project/MMU-Iris-Database/'

for root, dirs, files in os.walk(folder_path):
    for file in files:
        # Get path of image file
        eye_image_path = os.path.join(root, file).replace('\\', '/')
        print(eye_image_path)
        temp = ' '.join(eye_image_path.split('/')[1:4])
        
        # Load Image and file details
        eye_image_gs, eye_folder_path, eye_filename = load_image(eye_image_path)

        # Pupil Segmentation
        
        threshold = 0.9
        pupil_circle_x, pupil_circle_y, pupil_circle_radius = pupil_segmentation(eye_image_gs, threshold)
        
        while pupil_circle_x - pupil_circle_radius < 0 or pupil_circle_x + pupil_circle_radius > eye_image_gs.shape[1] or pupil_circle_y - pupil_circle_radius < 0 or pupil_circle_y + pupil_circle_radius > eye_image_gs.shape[0]:
            threshold -= 0.1
            if threshold > 0.5:
                pupil_circle_x, pupil_circle_y, pupil_circle_radius = pupil_segmentation(eye_image_gs, threshold)
            else:
                print(f"__________________________________Pupil Segmentation error {temp}")
            
        # Iris Segmentation
        
        gamma = 3
        circles = gamma_correction(gamma, eye_image_gs, pupil_circle_radius)
        
        # while circles is None:
        #     if threshold > 0.5:
        #         threshold -= 0.1
        #         pupil_circle_x, pupil_circle_y, pupil_circle_radius = pupil_segmentation(eye_image_gs, threshold)
        #         circles = gamma_correction(gamma, eye_image_gs, pupil_circle_radius)
        #     else:
        #         print(f"******************************************Iris Segmentation Error {temp}")
        
        iris_circle = iris_seg(gamma, eye_image_gs, pupil_circle_x, pupil_circle_y, pupil_circle_radius, threshold, circles)
        
        # Iris Normalization
        
        height = 50
        width = 360
        eye_image_rubber = iris_normalization(eye_image_gs, iris_circle, pupil_circle_radius, height, width)
        
        # Eyelash Detection
        
        threshold_ed = 0.7
        eye_image_rubber_inv_dil = eyelash_detection(eye_image_rubber, threshold_ed)
        
        # Parabolic Projection
        
        eye_image_rubber_parablic_projection = parabolic_projection(eye_image_rubber_inv_dil, eye_image_rubber)

        # Histogram equalized parabolic projection

        finalIMG = histogram_equalization(eye_image_rubber_parablic_projection)

        print(f"{temp} Done")